In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('./attacks.csv',sep=',', encoding='latin-1')

In [3]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.columns = df.columns.to_series().apply(lambda x: x.strip())

In [6]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
df = df.drop_duplicates()

In [8]:
df.shape

(6312, 24)

In [9]:
df = df.rename(columns={'Case Number':'Case_number', 'Fatal (Y/N)':'Criticality','Investigator or Source':'Investigator', 'href formula':'href1', 'Case Number.1':'Case_number1', 'Case Number.2':'Case_number2', 'original order':'Order', 'Unnamed: 22':'Desconocido1', 'Unnamed: 23':'Desconocido2'})

In [10]:
df.columns

Index(['Case_number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Criticality', 'Time',
       'Species', 'Investigator', 'pdf', 'href1', 'href', 'Case_number1',
       'Case_number2', 'Order', 'Desconocido1', 'Desconocido2'],
      dtype='object')

In [11]:
df = df.rename(columns={'Injury':'Hurt', 'Criticality':'Lethal'})

In [12]:
df.head()

,Case_number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator,pdf,href1,href,Case_number1,Case_number2,Order,Desconocido1,Desconocido2
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [13]:
df = df.rename(columns={'Type':'Type_attack'})

In [14]:
df.head()

,Case_number,Date,Year,Type_attack,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator,pdf,href1,href,Case_number1,Case_number2,Order,Desconocido1,Desconocido2
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [15]:
df.shape

(6312, 24)

In [16]:
df.isna().sum()

Case_number        2
Date              10
Year              12
Type_attack       14
Country           60
Area             465
Location         550
Activity         554
Name             220
Sex              575
Age             2841
Hurt              38
Lethal           549
Time            3364
Species         2848
Investigator      27
pdf               10
href1             11
href              10
Case_number1      10
Case_number2      10
Order              3
Desconocido1    6311
Desconocido2    6310
dtype: int64

In [17]:
nan_columns = df.isna().sum()

In [18]:
nan_columns [nan_columns > 0]

Case_number        2
Date              10
Year              12
Type_attack       14
Country           60
Area             465
Location         550
Activity         554
Name             220
Sex              575
Age             2841
Hurt              38
Lethal           549
Time            3364
Species         2848
Investigator      27
pdf               10
href1             11
href              10
Case_number1      10
Case_number2      10
Order              3
Desconocido1    6311
Desconocido2    6310
dtype: int64

In [19]:
df[['Case_number','Date','Year','Type_attack','Country','Hurt','Investigator','pdf','href1','href','Case_number1','Case_number2','Order']] = df[['Case_number','Date','Year','Type_attack','Country','Hurt','Investigator','pdf','href1','href','Case_number1','Case_number2','Order']].fillna('Unknown')

In [20]:
nan_columns [nan_columns > 0]

Case_number        2
Date              10
Year              12
Type_attack       14
Country           60
Area             465
Location         550
Activity         554
Name             220
Sex              575
Age             2841
Hurt              38
Lethal           549
Time            3364
Species         2848
Investigator      27
pdf               10
href1             11
href              10
Case_number1      10
Case_number2      10
Order              3
Desconocido1    6311
Desconocido2    6310
dtype: int64

In [21]:
df.isna().sum()

Case_number        0
Date               0
Year               0
Type_attack        0
Country            0
Area             465
Location         550
Activity         554
Name             220
Sex              575
Age             2841
Hurt               0
Lethal           549
Time            3364
Species         2848
Investigator       0
pdf                0
href1              0
href               0
Case_number1       0
Case_number2       0
Order              0
Desconocido1    6311
Desconocido2    6310
dtype: int64

In [22]:
df.shape

(6312, 24)

In [23]:
df = df.drop (df[df.Case_number == 'Unknown'].index, axis=0)

In [24]:
df = df.drop (df[df.Case_number == 'Unknown'].index, axis=0)
df = df.drop (df[df.Date == 'Unknown'].index, axis=0)
df = df.drop (df[df.Year == 'Unknown'].index, axis=0)
df = df.drop (df[df.Type_attack == 'Unknown'].index, axis=0)
df = df.drop (df[df.Country == 'Unknown'].index, axis=0)
df = df.drop (df[df.Hurt == 'Unknown'].index, axis=0)
df = df.drop (df[df.Investigator == 'Unknown'].index, axis=0)
df = df.drop (df[df.pdf == 'Unknown'].index, axis=0)
df = df.drop (df[df.href1 == 'Unknown'].index, axis=0)
df = df.drop (df[df.Case_number1 == 'Unknown'].index, axis=0)
df = df.drop (df[df.Case_number2 == 'Unknown'].index, axis=0)
df = df.drop (df[df.Order == 'Unknown'].index, axis=0)




In [25]:
df.isna().sum()

Case_number        0
Date               0
Year               0
Type_attack        0
Country            0
Area             411
Location         492
Activity         522
Name             199
Sex              545
Age             2752
Hurt               0
Lethal           520
Time            3273
Species         2772
Investigator       0
pdf                0
href1              0
href               0
Case_number1       0
Case_number2       0
Order              0
Desconocido1    6201
Desconocido2    6200
dtype: int64

In [26]:
df.shape

(6202, 24)

In [27]:
nan_columns = df.isna().sum()
nan_columns [nan_columns > 0]

Area             411
Location         492
Activity         522
Name             199
Sex              545
Age             2752
Lethal           520
Time            3273
Species         2772
Desconocido1    6201
Desconocido2    6200
dtype: int64

In [28]:
df.columns

Index(['Case_number', 'Date', 'Year', 'Type_attack', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex', 'Age', 'Hurt', 'Lethal', 'Time',
       'Species', 'Investigator', 'pdf', 'href1', 'href', 'Case_number1',
       'Case_number2', 'Order', 'Desconocido1', 'Desconocido2'],
      dtype='object')

In [29]:
df.head()

,Case_number,Date,Year,Type_attack,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator,pdf,href1,href,Case_number1,Case_number2,Order,Desconocido1,Desconocido2
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,NaN,NaN
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,NaN,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,NaN,NaN
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,NaN,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,NaN,NaN


In [30]:
df.Date.apply(lambda x: re.findall('\w+',x)).head(20)

0               [25, Jun, 2018]
1               [18, Jun, 2018]
2               [09, Jun, 2018]
3               [08, Jun, 2018]
4               [04, Jun, 2018]
5               [03, Jun, 2018]
6               [03, Jun, 2018]
7               [27, May, 2018]
8               [26, May, 2018]
9               [26, May, 2018]
10              [24, May, 2018]
11              [21, May, 2018]
12              [13, May, 2018]
13              [13, May, 2018]
14                  [May, 2018]
15              [12, May, 2018]
16              [09, May, 2018]
17    [Reported, 30, Apr, 2018]
18              [28, Apr, 2018]
19              [28, Apr, 2018]
Name: Date, dtype: object

In [31]:
df.Date

0       25-Jun-2018
1       18-Jun-2018
2       09-Jun-2018
3       08-Jun-2018
4       04-Jun-2018
           ...     
6297    Before 1903
6298    Before 1903
6299      1900-1905
6300      1883-1889
6301      1845-1853
Name: Date, Length: 6202, dtype: object

In [32]:
df.shape

(6202, 24)

In [33]:
df.Date.unique()

array(['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', ..., '1900-1905',
       '1883-1889', '1845-1853'], dtype=object)

In [34]:
df

,Case_number,Date,Year,Type_attack,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator,pdf,href1,href,Case_number1,Case_number2,Order,Desconocido1,Desconocido2
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,NaN,NaN
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,NaN,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,NaN,NaN
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,NaN,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
6298,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
6299,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
6300,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6202 entries, 0 to 6301
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Case_number   6202 non-null   object
 1   Date          6202 non-null   object
 2   Year          6202 non-null   object
 3   Type_attack   6202 non-null   object
 4   Country       6202 non-null   object
 5   Area          5791 non-null   object
 6   Location      5710 non-null   object
 7   Activity      5680 non-null   object
 8   Name          6003 non-null   object
 9   Sex           5657 non-null   object
 10  Age           3450 non-null   object
 11  Hurt          6202 non-null   object
 12  Lethal        5682 non-null   object
 13  Time          2929 non-null   object
 14  Species       3430 non-null   object
 15  Investigator  6202 non-null   object
 16  pdf           6202 non-null   object
 17  href1         6202 non-null   object
 18  href          6202 non-null   object
 19  Case_n

In [36]:
nan_columns = df.isna().sum()
nan_columns [nan_columns > 0]

Area             411
Location         492
Activity         522
Name             199
Sex              545
Age             2752
Lethal           520
Time            3273
Species         2772
Desconocido1    6201
Desconocido2    6200
dtype: int64

In [38]:
df.to_csv('/Users/juan/desktop/data_clean/Attacks-Clean.csv')